In [ ]:
import tempfile
from pathlib import Path

from PIL import Image
import numpy as np
import matplotlib.pylab as plt

from utils import load_cifar10, batch_plot, seed_everything, moving_average, weights_to_images


seed_everything()
np.set_printoptions(precision=3)

%load_ext autoreload
%autoreload 2

benchmark = False
save_weights_update = False

In [ ]:
(X_train, y_train), (X_test, y_test) = load_cifar10("../code/cs231n/datasets/cifar-10-batches-py/")

assert X_train.shape == (50000, 32, 32, 3)
assert y_train.shape == (50000,)
assert X_test.shape == (10000, 32, 32, 3)
assert y_test.shape == (10000,)

In [ ]:
# 0: airplane
# 1. automobile
# 2: bird
# 3: cat
target_classes = [0, 1, 2, 3]
train_indices = np.isin(y_train, target_classes)
test_indices = np.isin(y_test, target_classes)

X_train, y_train = X_train[train_indices], y_train[train_indices]
X_test, y_test = X_test[test_indices], y_test[test_indices]

In [ ]:
# Get the indexes of 'batch_size' random digits
batch_size = 16
random_indexes = np.random.randint(X_train.shape[0], size=batch_size)
# Plot digits with labels
batch_plot(X_train[random_indexes], y_train[random_indexes], with_border=False)

In [ ]:
tmp_dir = Path(tempfile.mkdtemp())
sample_images = {"cat": 110, "bird": 314, "airplane_0": 153, "automobile": 49, "airplane_1": 25}

for c, i in sample_images.items():
    Image.fromarray(X_train[i]).save(tmp_dir.joinpath(f"image_{c}.png"))

In [ ]:
weights = np.random.randn(len(target_classes), *X_train[0].shape) * 0.0001
bias = np.zeros((len(sample_images), 1))

weights_image = weights_to_images(weights)

for i, w in enumerate(weights_image):
    Image.fromarray(w).resize((128, 128)).save(tmp_dir.joinpath(f"weights_{i}_epoch_0000.png"))

In [42]:
!open {tmp_dir}

In [ ]:
s = np.einsum("nhwc,mhwc->nm", X_train[list(sample_images.values())], weights) + bias
y = y_train[list(sample_images.values())]
sy = s[np.arange(len(y)), y]
margins = np.maximum(0, s - sy[:, None] + 1)
loss = np.sum(margins) / len(y) - 1

assert s.shape == (len(sample_images), len(target_classes))
assert np.allclose(y, [3, 2, 0, 1, 0])
assert margins.shape == (len(sample_images), len(target_classes))

<img src="images/svm_training_epoch_0000.png">

In [ ]:
from svm import svm_loss_original, svm_loss_reshaped, svm_loss_original_with_grads, svm_loss_reshaped_with_grads


reg = 1e3
W = np.random.randn(len(target_classes), *X_train[0].shape) * 0.0001
W_reshaped, X_train_reshaped = W.reshape((len(target_classes), -1)).T, X_train.reshape((len(y_train), -1))
bias = np.zeros((len(X_train), 1))

if benchmark:
    %timeit svm_loss_original(x=X_train, y=y_train, weights=W, bias=bias, reg=reg)
    %timeit svm_loss_reshaped(x=X_train_reshaped, y=y_train, weights=W_reshaped, bias=bias, reg=reg)

assert np.allclose(
    svm_loss_original(x=X_train, y=y_train, weights=W, bias=bias, reg=reg),
    svm_loss_reshaped(x=X_train_reshaped, y=y_train, weights=W_reshaped, bias=bias, reg=reg),
)

In [ ]:
if benchmark:
    %timeit svm_loss_original_with_grads(x=X_train, y=y_train, weights=W, bias=bias, reg=reg)
    %timeit svm_loss_reshaped_with_grads(x=X_train_reshaped, y=y_train, weights=W_reshaped, bias=bias, reg=reg)

loss_original, dw_original, db_original = svm_loss_original_with_grads(
    x=X_train, y=y_train, weights=W, bias=bias, reg=reg
)
loss_reshaped, dw_reshaped, db_reshaped = svm_loss_reshaped_with_grads(
    x=X_train_reshaped, y=y_train, weights=W_reshaped, bias=bias, reg=reg
)

assert np.allclose(loss_original, loss_reshaped), "loss not equal"
assert np.allclose(dw_original, dw_reshaped.T.reshape(dw_original.shape)), "delta of weights not equal"
assert np.allclose(db_original, db_reshaped), "delta of bias not equal"

In [ ]:
mean_image = np.mean(X_train, axis=0)

X_train = X_train.astype(float) - mean_image
X_test = X_test.astype(float) - mean_image

In [ ]:
from svm import train, predict


history, weights, bias = train(
    X_train, y_train, reg=4e3, learning_rate=3e-7, num_iters=3000, batch_size=200, verbose=True
)

pred_train = predict(X_train, weights, bias)
pred_valid = predict(X_test, weights, bias)
train_accuracy = np.mean(y_train == pred_train)
valid_accuracy = np.mean(y_test == pred_valid)

print(f"train acc: {train_accuracy:.4f}, valid acc: {valid_accuracy:.4f}")

In [ ]:
plt.plot(history["loss"], color="gray", alpha=0.8)
plt.plot(moving_average(history["loss"]), color="k")
plt.grid("on")
plt.show()

In [ ]:
if save_weights_update:
    import io
    from PIL import Image

    total_weights = []
    update_steps = 30

    for weights in history["weights"][::update_steps]:
        weights_image = weights_to_images(weights.reshape((-1, 32, 32, 3)))

        img_buf = io.BytesIO()
        batch_plot(weights_image, with_border=False, save_path=img_buf)
        total_weights.append(Image.open(img_buf).resize((512, 512)).copy())
        img_buf.close()

    total_weights[0].save(
        f"images/svm_weights_update.gif",
        save_all=True,
        append_images=total_weights[1:],
        optimize=False,
        duration=100,
        loop=0,
    )

In [ ]:
weights, bias = history["weights"][-1], history["bias"][-1]
weights_image = weights_to_images(weights)
batch_plot(weights_image)

<img src="images/svm_weights_update.gif" width="800">

In [ ]:
s = np.einsum("nhwc,mhwc->nm", X_train[list(sample_images.values())], weights) + bias
y = y_train[list(sample_images.values())]
sy = s[np.arange(len(y)), y]
margins = np.maximum(0, s - sy[:, None] + 1)
loss = np.sum(margins) / len(y) - 1

<img src="images/svm_training_epoch_2950.png">